In [ ]:
#!pip install python-dotenv

# TODO
- Odd one out with numbers of answers
- [urgent] Word topic matching is very wrong format -> do numbers make sense here?
- There should be free text answers!
- [urgent] for gap text, implement free text answers more clearly. sometimes the gap number repeats
  - test the free text answer formatting here
  - sometimes provides correct answers in the correct order
- code the level upgrade in python logic
- code the level assessment somehow -> there should be a way to input the language level

In [23]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib

importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [24]:
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



In [27]:
profile = await db.load_profile(id='Franzy')
domain = await ui.choose_domain(domain='text')
task_type = await ui.choose_task_type(task_type='odd_one_out')
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [28]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate a task description, and the task according to the format of the examples. No other text or tasks.
Generate a odd one out, provide an enumerated list of 5 words, where one does not fit in. Give only a list of newly generated words according to the topic and a task description. Do not give the correct answer. Generate it for the area of text teaching, targeting on media.
Adjust the task dificulty to language level C2 (Can understand with ease virtually everything heard or read. Can summarise information from different spoken and written sources, reconstructing arguments and accounts in a coherent presentation. Can express him/herself spontaneously, very fluently and precisely, differentiating finer shades of meaning even in more complex situations).


In [36]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Task description: 
In this task, you will be presented with a list of five words related to the topic of media. One of the words does not fit in with the others. Your task is to identify the odd one out and provide a justification for your choice. Make sure to consider the meaning and usage of each word in the context of media.

List of words:

1. Newspaper
2. Television
3. Social media
4. Magazine
5. Refrigerator


In [32]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task, answer="television doesn't fit in")
formated_user_answer = await formater.learner_answer_formatting(user_answer)


In [33]:
######################
# This is what it will actually looks like
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

NL_feedback = Great job! Your answer is correct. All of the other words - newspaper, social media, magazine, and camera - are all forms of media that can be used to communicate and share information. Keep up the good work!
correctness = correct
training_goals = no training goals


In [25]:
# While the parsing is not implemented, this function will not work
#new_profile = await evaluator.update_learner_profile(training_goals, profile)
#print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

In [ ]:
# This overwrites the original profile
# For testing purposes, it's easier to just comment it out
# await db.save_profile(new_profile)